# Design, Train and Test a ResNet 18 model from scratch on CIFAR10 dataset. Plot the performance metrics and evaluate the trained model on the test set by visualization.

In [ ]:
# Reimport necessary libraries after execution state reset
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

# Load CIFAR-10 dataset and create small subsets 60000 
batch_size = 32  # Minimal batch size for memory efficiency default = 32
subset_size = 5000  # Use a subset of CIFAR-10 for training

# Download CIFAR-10
full_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
full_testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create small subsets for training and testing
train_subset, _ = random_split(full_trainset, [subset_size, len(full_trainset) - subset_size])
test_subset, _ = random_split(full_testset, [subset_size // 2, len(full_testset) - subset_size // 2])

# Create dataloaders
trainloader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=0)
testloader = DataLoader(test_subset, batch_size=batch_size, shuffle=False, num_workers=0)

# Define ResNet-18 model
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        identity = self.shortcut(x)
        out = self.conv1(x)
        out = self.bn1(out)
        out = torch.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity
        out = torch.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, num_blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = torch.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

# Create ResNet-18 model
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

# Initialize model, loss, and optimizer
model = ResNet18().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.05)

# Train the model with reduced memory usage
num_epochs = 100  # Reduce epochs to prevent memory overload
train_losses, test_losses = [], []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_loss = running_loss / len(trainloader)
    train_losses.append(train_loss)

    # Evaluate on test set
    model.eval()
    test_loss = 0.0
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_losses.append(test_loss / len(testloader))
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_losses[-1]:.4f}, Test Acc: {100 * correct / total:.2f}%")

# Save model checkpoint
torch.save(model.state_dict(), "resnet18_cifar10_small.pth")

# Plot Training and Testing Loss
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training vs. Testing Loss")
plt.legend()
plt.show()

# Proceed to evaluation separately to optimize memory usage

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/100], Train Loss: 2.6889, Test Loss: 2.1779, Test Acc: 17.52%
Epoch [2/100], Train Loss: 2.1230, Test Loss: 2.0414, Test Acc: 23.24%
Epoch [3/100], Train Loss: 2.0477, Test Loss: 2.0148, Test Acc: 19.24%
Epoch [4/100], Train Loss: 1.9413, Test Loss: 1.8827, Test Acc: 27.68%
Epoch [5/100], Train Loss: 1.8430, Test Loss: 1.7864, Test Acc: 33.00%
Epoch [6/100], Train Loss: 1.7652, Test Loss: 1.6888, Test Acc: 36.28%
Epoch [7/100], Train Loss: 1.6917, Test Loss: 1.7215, Test Acc: 34.04%
Epoch [8/100], Train Loss: 1.6128, Test Loss: 1.5891, Test Acc: 40.28%
Epoch [9/100], Train Loss: 1.5493, Test Loss: 1.6206, Test Acc: 39.08%
Epoch [10/100], Train Loss: 1.5028, Test Loss: 1.5973, Test Acc: 42.00%
Epoch [11/100], Train Loss: 1.3934, Test Loss: 1.4885, Test Acc: 44.24%
Epoch [12/100], Train Loss: 1.3347, Test Loss: 1.4727, Test Acc: 45.72%
Epoch [13/100], Train Loss: 1.2689, Test Loss: 1.4162, Test Acc: 47.3

# Increase test accuracy to 90%
# Avoid Overfitting
# Print Classification report
# Visualize the test results 